<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Extracting-tweets-from-SQL-database" data-toc-modified-id="Extracting-tweets-from-SQL-database-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting tweets from SQL database</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Word2vec</a></span></li></ul></li></ul></li><li><span><a href="#Topic-Modeling" data-toc-modified-id="Topic-Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Topic Modeling</a></span></li></ul></div>

# Libraries 

    The following Libraries were used in the development of this project:

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
import nltk
import pyodbc
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
import json 

import nltk 


'''Natural Language Processing libraries'''
import nltk 
import gensim
import regex as re
import spacy
from spacy.lang.en import English
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
import re, string, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from gensim.models import FastText
from gensim.models import Word2Vec

import preprocessor as p

import warnings
warnings.filterwarnings("ignore")

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools

import matplotlib.pyplot as plt

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')




[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\afabi/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\afabi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\afabi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Preprocessing

The data preprocessing consist of different stages:

In [27]:
class Document:
    """ Retrieve the narratives from the DataFrame and respectively
        store and pre-process it. 
        
        :param df: DataFrame including the reports and the predictor variable. 
        
        
        :ivar data: Stores the DataFrame.
        :ivar text: Stores the narratives as string.
        :ivar corpus: Stores the pre-processed text.
    """
    
    
    def __init__(self, df):
        self.data = df
        self.text = df["text"].astype(str)
        self.textPreProcessing()
        
        
    def remove_non_ascii(self, words):
        """Remove non-ASCII characters from list of tokenized words
        
        :param words:  List of words to be transformed when removing non_ascii characters.
        
        :return new_words: List of words after the transformation of removed non_ascii characters.
        
        """
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words


    def remove_punctuation(self, words):
        """Remove punctuation from list of tokenized words
        
        :param words:  List of words that will get remove their punctuations, if any. 
        
        :return new_words: List of transformed words.
        
        
        """
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words


    def stem_words(self, words):
        """Stem words in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective stems.
        
        
        """
        
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems
    
    def lemmatize_verbs(self, words):
        """Lemmatize verbs in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective lemmas.
        
        """
        
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas
    
    
    
    def remove_stopwords(self, words):
        """Remove common words that have no meaning or importance in the sentence.

        :param words:  List of words to be processed and get stop words removed.. 

        :return new_words: List of words with the stop words already removed."""
            
        
        stop_words = set(stopwords.words('spanish')) 
        stop_words1 = set(stopwords.words('english')) 


        
        for word in stop_words:
            if word in words:
                words.remove(word)
                
        for word in stop_words1:
            if word in words:
                words.remove(word)
                
        return words


    
    def normalize(self, words):
        words = self.remove_non_ascii(words)
        words = self.remove_stopwords(words)
        words = self.remove_punctuation(words)
        words = self.lemmatize_verbs(words)
        return words
    
    
    def textPreProcessing(self):
        """Pre-process the text, normalize and clean it.
        The function stores the cleaned text in the self.data
        attribute. """
        p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)


        clean_text = []

        for narrative in self.text:
            sentence = p.clean(narrative)
            #sentence = re.sub('RT @[\w_]+:', '', sentence)

            #sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
            sentence = word_tokenize(sentence)
            sentence = self.normalize(sentence)
                
                
            clean_text.append(sentence)
            
            
                    
        print(len(self.text), len(clean_text))
        self.data["clean_text"] = clean_text

In [26]:
def concatTweets(table):
    tweets = ''
    c = 0
    for row in df.tweets:
        if(c == 0):
            c+= 1
            tweets = pd.DataFrame(json.loads(row))
        else:
            tweets = pd.concat([tweets, pd.DataFrame(json.loads(row))], axis=0, ignore_index=True)
    
    
    return tweets 

# Extracting tweets from SQL database

In [18]:
server = 'sqldatamining.database.windows.net'
database = 'SNA'
username = 'UserAdmin'
password = 'Machomen123'   
driver= '{ODBC Driver 13 for SQL Server}'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=sqldatamining.database.windows.net,1433', user='UserAdmin' , password='Machomen123', database='SNA')

In [19]:
cursor = cnxn.cursor()

In [36]:
rows = cursor.execute("SELECT * from dbo.Users1 ").fetchmany(10)

In [37]:
df = pd.DataFrame((tuple(t) for t in rows), columns=['id', 
 'author',
 'created_at',
 'location',
 'description',
 'verified',
 'followers',
 'following',
 'favourites_count',
 'statuses_count',
 'lang',
 'tweets',
 'following_json',
 'followers_json']) 


In [38]:
df.head(40)

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,2284857094,themoonisironic,2014-01-16 01:10:33,‘97 // they/them // ????????,"where there is hope, there are trials // art: ...",False,101,516,121166,36377,None,"[{""id"": 1356030533780824069, ""created_at"": ""20...","[{""id"": 1387079251476815875, ""author"": ""rikuno...","[{""id"": 1384297303129825284, ""author"": ""bts123..."
1,834324620,bbelita23,2012-09-19 23:21:48,"spooky town, PR",just put it out into the universe,False,670,443,33428,92917,None,"[{""id"": 1356026926134140928, ""created_at"": ""20...","[{""id"": 2513536418, ""author"": ""Kvn_Astacio"", ""...","[{""id"": 1289669319526420482, ""author"": ""keviin..."
2,175521394,Stormfocus18,2010-08-06 21:03:08,Puerto Rico_Island,"Loving #MewGulf and #KristSingto, amante de la...",False,199,187,6497,38945,None,[],"[{""id"": 1387677517822054400, ""author"": ""tinest...","[{""id"": 1387677517822054400, ""author"": ""tinest..."
3,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
4,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]
5,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
6,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]
7,908687401123553280,rsantanafonseca,2017-09-15 13:42:13,"COPU, UPRRP","19 (+3) | ?????? | Pop culture enthusiast, soc...",False,891,620,121676,124488,None,[],"[{""id"": 755882683, ""author"": ""_vidalysrms"", ""c...","[{""id"": 1344047685716815872, ""author"": ""LeQuee..."
8,1346688657365889024,SJCiudadCapital,2021-01-06 05:23:04,,Cuenta Oficial de la Ciudad Capital de Puerto ...,False,2915,183,291,1157,None,"[{""id"": 1355954464436473856, ""created_at"": ""20...","[{""id"": 1309256724830969857, ""author"": ""estefa...","[{""id"": 1271152649304518657, ""author"": ""JCruz_..."
9,521836439,Angelicv_G,2012-03-12 01:12:16,Puerto Rico,UT • yo no sé que poner aquí??,False,962,519,115665,130883,None,"[{""id"": 1356041673227313154, ""created_at"": ""20...","[{""id"": 1192074658994180096, ""author"": ""Nsnili...","[{""id"": 1383464766283280389, ""author"": ""geegee..."


In [39]:
index = 0
for tweets in df.tweets:
    print(len(tweets))
    if(len(tweets) == 2):
        df.drop([index], inplace=True, axis=0)
    
    index+=1
    
df.reset_index(inplace=True)

index = 0
temp_user = ""
for user in df.author:
    if(user != temp_user):
        print("a")
        temp_user = user 
    else:
        df.drop([index], inplace=True, axis=0)
    
    
    index+=1

435940
181519
2
34422
2
34422
2
2
68260
331733
a
a
a
a
a


In [24]:
df.head(10)

,index,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,0,2284857094,themoonisironic,2014-01-16 01:10:33,‘97 // they/them // ????????,"where there is hope, there are trials // art: ...",False,101,516,121166,36377,None,"[{""id"": 1356030533780824069, ""created_at"": ""20...","[{""id"": 1387079251476815875, ""author"": ""rikuno...","[{""id"": 1384297303129825284, ""author"": ""bts123..."


In [45]:
x = concatTweets(df)


x.head()

,id,created_at,text,coordinates,geo,user,retweet_count,favorite_count
0,1356030533780824069,2021-02-01T00:04:07,RT @geheichou: Childe I told you to stop\n#Gen...,None,None,themoonisironic,4929,0
1,1355996223753383939,2021-01-31T21:47:47,i’m also obsessed with a curse stemming from l...,None,None,themoonisironic,0,0
2,1355995462784978944,2021-01-31T21:44:46,i also really like yuta - he’s sweet but terri...,None,None,themoonisironic,0,0
3,1355995461820137472,2021-01-31T21:44:45,"kk, finished jjk 0\nit was really interesting ...",None,None,themoonisironic,0,0
4,1355991939976454146,2021-01-31T21:30:46,"@hinosreis no, jujutsu kaisen!! it’s off next ...",None,None,themoonisironic,0,1


### Word2vec

In [356]:
model = Word2Vec(documento.data["clean_text"], min_count=0, workers=20, window=2,  alpha=0.02, hs=1)



In [503]:
my_dict = dict({})
for idx, key in enumerate(model.wv.key_to_index):
    my_dict[key] = model.wv[key]

In [504]:
vectores = []
for a in documento.data.clean_text:
    t  = []
    for word in a:
        try:
            t.append(my_dict[word])
        except:
            continue
    
    vectores.append(t)
        
documento.data["vectorized_text"] = vectores
documento.data.tail()

,text,clean_text,vectorized_text
113,": El ajusta la magnitud a , en el límite de pr...","[El, ajusta, magnitud, limite, producir, tsunami]","[[-0.017272485, 0.008652826, 0.007515971, 0.00..."
114,: “ Van a seguir ocurriendo eventos fuertes. H...,"[Van, seguir, ocurriendo, eventos, fuertes, Ha...","[[-0.0027897793, -0.0016228552, -0.007687815, ..."
115,: Casa colapsada en Yauco.,"[Casa, colapsada, Yauco]","[[0.008535036, 0.013252284, 0.00035941493, 0.0..."
116,: “Las réplicas podrían ser de magnitud mayor ...,"[Las, replicas, podrian, ser, magnitud, mayor,...","[[0.0029013832, -0.0027108788, -0.0015446608, ..."
117,": PRELIMINAR -01-06 :58:03 No hay Aviso, Adver...","[PRELIMINAR, 06, 5803, No, Aviso, Advertencia,...","[[-0.0127012255, 0.0040803654, 0.009027934, -0..."


In [505]:
df.tail()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
5,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
6,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]
7,908687401123553280,rsantanafonseca,2017-09-15 13:42:13,"COPU, UPRRP","19 (+3) | ?????? | Pop culture enthusiast, soc...",False,891,620,121676,124488,None,[],"[{""id"": 755882683, ""author"": ""_vidalysrms"", ""c...","[{""id"": 1344047685716815872, ""author"": ""LeQuee..."
8,1346688657365889024,SJCiudadCapital,2021-01-06 05:23:04,,Cuenta Oficial de la Ciudad Capital de Puerto ...,False,2915,183,291,1157,None,"[{""id"": 1355954464436473856, ""created_at"": ""20...","[{""id"": 1309256724830969857, ""author"": ""estefa...","[{""id"": 1271152649304518657, ""author"": ""JCruz_..."
9,521836439,Angelicv_G,2012-03-12 01:12:16,Puerto Rico,UT • yo no sé que poner aquí??,False,962,519,115665,130883,None,"[{""id"": 1356041673227313154, ""created_at"": ""20...","[{""id"": 1192074658994180096, ""author"": ""Nsnili...","[{""id"": 1383464766283280389, ""author"": ""geegee..."


# Topic Modeling 

In [33]:
def asignTopics(users, tweetss):
    topics = userSubSets(tweetss)
   
    
  
    
    
    return users






def userSubSets(tweets_df):
    user = ''
    topics = []
    for instance in tweets_df.user:
        if(user != instance):
            print(instance)
            user = instance
            actual_user =  tweets_df['user']== user
            user_subset = tweets_df[actual_user]
            topics.append(topicModeling(user_subset))
        else:
            continue
            
            
    return topics
            

            
            
def topicModeling(user_subset):
    tweets = Document(pd.DataFrame(user_subset))
    #id2word = corpora.Dictionary(tweets.data.clean_text)

    # Create Corpus
    #texts = tweets.data.clean_text

    # Term Document Frequency
    #corpus = [id2word.doc2bow(text) for text in texts]
    
    covid = '''covid pandemia coronavirus vacuna cuarentena virus mascara propagacion'''
    
    string1 = covid
    words = string1.split()
    covid = " ".join(sorted(set(words), key=words.index))
    
    
    e_scores = get_scores(covid, tweets.data.clean_text)
    print(e_scores)
    
    
    
    
    

def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)




def get_scores(group,tweets):
    scores = []
    for tweet in tweets:
        s = jaccard_similarity(group, tweet)
        scores.append(s)
    return scores


    
    
            
        
        
    

In [41]:
tweets = Document(x)

covid = '''covid pandemia coronavirus vacuna cuarentena virus mascara propagacion'''
politica = '''gobierno politica ppd pnp pip independentista popular wanda pierluisi gobernador senado camara legislador proyecto '''
emociones = '''depresion ansiedad felicidad emocion salud mental tristesa triste alegre contento ansioso'''
eventos = '''terremoto tsunami huracan lluvia calor tormenta '''

string1 = covid
words = string1.split()
covid = " ".join(sorted(set(words), key=words.index))

string1 = politica
words = string1.split()
politica = " ".join(sorted(set(words), key=words.index))

string1 = emociones
words = string1.split()
emociones = " ".join(sorted(set(words), key=words.index))

string1 = eventos
words = string1.split()
eventos = " ".join(sorted(set(words), key=words.index))


covid_scores = get_scores(covid, tweets.data.clean_text)
politica_scores = get_scores(politica, tweets.data.clean_text)
emociones_scores = get_scores(emociones, tweets.data.clean_text)
eventos_scores = get_scores(eventos, tweets.data.clean_text)


3773 3773


In [53]:
# create a jaccard scored df.
data  = {'names':x.user.to_list(),       'covid_scores':covid_scores,
         'politica_scores': politica_scores, 'emociones_scores':emociones_scores, 'eventos_scores':eventos_scores}
scores_df = pd.DataFrame(data)
#assign classes based on highest score
def get_classes(l1, l2, l3, l4):
    econ = []
    socio = []
    cul = []
    heal = []
    for i, j, k, l in zip(l1, l2, l3, l4):
        m = max(i, j, k, l)
        if m == i:
            econ.append(1)
        else:
            econ.append(0)
        if m == j:
            socio.append(1)
        else:
            socio.append(0)        
        if m == k:
            cul.append(1)
        else:
            cul.append(0)  
        if m == l:
            heal.append(1)
        else:
            heal.append(0)   
            
    return econ, socio, cul, heal

print(scores_df.head())
l1 = scores_df.covid_scores.to_list()
l2 = scores_df.politica_scores.to_list()
l3 = scores_df.emociones_scores.to_list()
l4 = scores_df.eventos_scores.to_list()
econ, socio, cul, heal = get_classes(l1, l2, l3, l4)

data = {'name': scores_df.names.to_list(), 'economic':econ, 'social':socio, 'culture':cul, 'health': heal}
class_df = pd.DataFrame(data)
#grouping the tweets by username
new_groups_df = class_df.groupby(['name']).sum()
#add a new totals column
new_groups_df['total'] = new_groups_df['health'] + new_groups_df['culture'] + new_groups_df['social'] +  new_groups_df['economic']
#add a new totals row
new_groups_df.loc["Total"] = new_groups_df.sum()  

             names  covid_scores  politica_scores  emociones_scores  \
0  themoonisironic           0.0              0.0               0.0   
1  themoonisironic           0.0              0.0               0.0   
2  themoonisironic           0.0              0.0               0.0   
3  themoonisironic           0.0              0.0               0.0   
4  themoonisironic           0.0              0.0               0.0   

   eventos_scores  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  


In [54]:
new_groups_df.head()

,economic,social,culture,health,total
name,,,,,
Angelicv_G,1094,1091,1085,1167,4437
CoraimaINegron,109,109,109,118,445
SJCiudadCapital,198,197,195,208,798
bbelita23,688,690,686,713,2777
themoonisironic,1191,1182,1179,1517,5069


In [ ]:
import google.cloud 

from google.cloud import language_v1
from google.cloud.language_v1 import enums
from google.cloud.language_v1 import types



client = language.LanguageServiceClient()

document = types.Document(tweets.data.clean_text,
           type=enums.Document.Type.PLAIN_TEXT)

annotations = client.analyze_sentiment(document=document,
                                       encoding_type='UTF32', timeout=600)

sentences = []
for s in annotations.sentences:
    sentences.append((s.text.content, s.sentiment.score, s.sentiment.magnitude))

df = pd.DataFrame(sentences, columns=['sentence', 'score', 'magnitude'])